In [43]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas
import datetime
from datetime import datetime
import requests
import re
from selenium import webdriver
from selenium.webdriver import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


In [38]:

browser = webdriver.Chrome(ChromeDriverManager().install())

In [28]:
browser.get("https://www.i-d-s.com/dubai/wine/wines-by-country/france.html?p=2")   

In [169]:


parameters= [
{
"country": "TH",   
"website": "OIS",   
"url_scrap": (("france","https://www.i-d-s.com/dubai/wine/wines-by-country/france.html?p="),          
("argentina","https://www.i-d-s.com/dubai/wine/wines-by-country/argentina.html?p="),
("australia","https://www.i-d-s.com/dubai/wine/wines-by-country/australia.html?p="),
("chile","https://www.i-d-s.com/dubai/wine/wines-by-country/chile.html?p="),
("new_zealand","https://www.i-d-s.com/dubai/wine/wines-by-country/new-zealand.html?p="),
("south_africa","https://www.i-d-s.com/dubai/wine/wines-by-country/south-africa.html?p="),
("uk","https://www.i-d-s.com/dubai/wine/wines-by-country/united-kingdom.html?p="),
("usa","https://www.i-d-s.com/dubai/wine/wines-by-country/usa.html?p="),
("germany","https://www.i-d-s.com/dubai/wine/wines-by-country/germany.html?p="),
("italy","https://www.i-d-s.com/dubai/wine/wines-by-country/italy.html?p="),
("spain","https://www.i-d-s.com/dubai/wine/wines-by-country/spain.html?p="),
("rest-of-the-world","https://www.i-d-s.com/dubai/wine/wines-by-country/rest-of-the-world.html?p=")),
    
    
"price_regex":  "data-price-amount=\"(.*?)\"",
"url_product_regex": "class=\"product-item-link\" href=.*?>(.*?)<",                
"url_image_regex": "NA",
"brand_regex": "NA", 
"scrolling_type" : "no scroll",
"pop_up_type" : "have pop up", 
"page_not_found_type" : "yes",
"page_not_found_regex":"find products matching the selection",
"format_change_type" : "yes",
"multiple_pages" : "yes",
"scrapper_activated":"1"
}
    
]

DF_final=pandas.DataFrame()

In [160]:
def pop_up_closing(param,browser_param):
    if param["pop_up_type"]=='no pop up':
        pass
    if param["pop_up_type"]=='have pop up':

        close=browser_param.find_elements_by_class_name("action-close")
        for element in close : 
            try:
                element.click()   
                time.sleep(1)
            except:
                pass

In [195]:
def get_data(param, executable_path):
    
        l = len(param["url_scrap"])
        i=0
        df_aggregated=pandas.DataFrame()
        nb_issues = 0
        today_date=datetime.today().strftime('%Y-%m-%d')
        
        while i<l:
        #while i<2:
            
            #browser = webdriver.Chrome(executable_path=executable_path)
            browser = webdriver.Chrome(ChromeDriverManager().install())
            category=param["url_scrap"][i][0]           
            
            url=param["url_scrap"][i][1]
            print(category)
            #print(url)        

            # to change into a parameter
            test_not_found=param["page_not_found_regex"]

            #init
            page_not_found=0

            # different starting point depending on the param page not found 
            if param["page_not_found_type"]=='yes':
                page=1

            if param["page_not_found_type"]=='no':
                page=""


            while page_not_found==0:
                #use of try to handle possible scrapping errors or time outs - https://stackoverflow.com/questions/51943694/execute-when-error-occurs-python
                     
                url_with_page=url+str(page)
                print(url_with_page)     

                browser.get(url_with_page)    

                time.sleep(5)
                #use of the anti pop up function
                pop_up_closing(param,browser)

                #use of the scrolling function
                #scrolling(param,browser)

                #use of format changing
                #format_change(param,browser,page)

                # extract of sourc code 
                time.sleep(2)
                htmlsource = browser.page_source
                

                #use of the commas replace code
                #htmlsource_replaced = replace_commas(param,htmlsource)
                #print(htmlsource)

                time.sleep(2)

                prices = re.findall(param["price_regex"], htmlsource)
                print("prices: " + str(len(prices)))
                print(prices)
                
                url_product = re.findall(param["url_product_regex"], htmlsource)
                print("url_product: " + str(len(url_product)))
                print(url_product)
                
                df=[prices,url_product]
                df=pandas.DataFrame(df).transpose()
                df['country']=category
                df_aggregated=df_aggregated.append(df)


                # exit loop after first iteration if there are no pages to go through
                if param["page_not_found_type"]=='no':
                    page_not_found = 1

                if param["page_not_found_type"]=='yes':
                    page_not_found = len(re.findall(test_not_found, htmlsource))
                    page=page+1
                    # only keep the output if there is no shift 


            i=i+1
            browser.close()
             
            print("---------------")
        
        #define columns names

        return df_aggregated





In [196]:
df_wine_diplo=get_data(parameters[0],executable_path)

france
https://www.i-d-s.com/dubai/wine/wines-by-country/france.html?p=1
prices: 12
['42.05', '75.6', '148.85', '75.6', '3139.59', '1342.28', '15.55', '26.85', '25.35', '2025.288', '69.65', '49']
url_product: 12
['Domaine Cailbourdin, Pouilly-Sur-Loire, Pouilly-Fumé Les Cris ', 'Maison Antoine Geoffroy, Beaune, Gevrey Chambertin Vieilles Vignes ', 'Domaine Daniel-Etienne Defaix, Chablis Grand Cru Blanchot ', 'Domaine Daniel-Etienne Defaix, Chablis Vieilles Vignes', '2016 Château Duhart-Milon Rothschild, 4eme Cru Classé, Pauillac', '2019 "Brulieres" De Beychevelle, Haut-Medoc', 'Cave de Turckheim Gewurztraminer Reserve', 'Schröder and Schÿler Margaux Private Selection', "Perle d'Aurore, Crémant de Bourgogne Brut Rosé", '2015 Chateau Beychevelle "Amiral De Beychevelle" St. Julien', 'Remoissenet Pommard', 'Remoissenet Chablis 1er Cru Fourchaume']
https://www.i-d-s.com/dubai/wine/wines-by-country/france.html?p=2


/var/folders/9z/sjrbpv9j1996t3_4wv3ry65c0000gn/T/ipykernel_65512/2389928353.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aggregated=df_aggregated.append(df)


prices: 12
['30.35', '32.78', '7150', '52.65', '45.35', '29.95', '17', '10.15', '9.4', '12.45', '11.7', '45.75']
url_product: 12
['Remoissenet Chablis', 'Château Mont-Redon Châteauneuf-du-Pape Rouge', '2016 Château Angelus, 1er Grand Cru Classé, Saint Emilion', 'Perrier-Jouët Grand Brut', 'Savigny Les Beaune Antoine Geoffroy', 'Domaine Michel Girault Les Beaux Regards, Sancerre Rouge', 'M.Chapoutier, Rasteau " Les Gadilles"', 'M.Chapoutier, Costières-de-Nîmes, Collection Bio “Areca” &amp; “Rouge” ', "M.Chapoutier, Luberon 'La Ciboise' Rouge", 'M.Chapoutier, Côtes-du-Rhône Rouge Adubatio , Collection Bio', "M.Chapoutier, 'Belleruche' Côtes-du-Rhône Rouge", 'M.Chapoutier, Châteauneuf-du-Pape, Collection Bio Rouge']
https://www.i-d-s.com/dubai/wine/wines-by-country/france.html?p=3


/var/folders/9z/sjrbpv9j1996t3_4wv3ry65c0000gn/T/ipykernel_65512/2389928353.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aggregated=df_aggregated.append(df)


prices: 12
['12.15', '45.1', '27.15', '50.8', '12.45', '11.85', '9.4', '22.35', '17.1', '25.7', '21.2', '13.7']
url_product: 12
['Marius, Avenue de la Gare Syrah, IGP Collines Rhodaniennes ', "M.Chapoutier, Cornas 'Les Arènes'", "M.Chapoutier, Saint-Joseph 'Deschants' Rouge", "M.Chapoutier, Châteauneuf-du-Pape 'La Bernardine' Blanc", 'M.Chapoutier, Côtes-du-Rhône Blanc Adunatio , Collection Bio', "M.Chapoutier, 'Belleruche' Côtes-du-Rhône Blanc", "M.Chapoutier, Luberon 'La Ciboise' Blanc", "M.Chapoutier, Saint-Péray 'Les Tanneurs' ", "M.Chapoutier, 'La Combe Pilate' Viognier, IGP Collines Rhodaniennes", "M.Chapoutier, Saint-Joseph 'Deschants' Blanc", "Domaine Trénel, Climat Fleurie 'La Madone' (Single Vineyard)", 'Domaine Trénel, Beaujolais Organic " Rouge"']
https://www.i-d-s.com/dubai/wine/wines-by-country/france.html?p=4


...

prices: 11
['29.15', '22.2', '13.05', '22.2', '33.75', '34.8', '12.65', '11.5', '9.05', '6.8', '7.25']
url_product: 11
['Ktima Gerovassiliou AVATON', 'Teliani Valley Glekhuri Kisi Qvevri', 'Teliani Valley Saperavi', 'Teliani Valley Glekhuri Kisiskhevi Saperavi Qvevri', 'Grace Koshu Private Reserve, Grace Winery Grace Yamanashi', 'Grace Koshu Hishiyama, Private Reserve, Grace Winery, Yamanashi', 'Château Ksara Blanc de Blancs', 'Château Ksara Sunset Rosé', 'Vinho Verde Loureiro/Alvarinho, Azevedo, Portugal', 'Mateus Rosé', 'Terra Franca,Vinho Regional Tinto']
https://www.i-d-s.com/dubai/wine/wines-by-country/rest-of-the-world.html?p=3


/var/folders/9z/sjrbpv9j1996t3_4wv3ry65c0000gn/T/ipykernel_65512/2389928353.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aggregated=df_aggregated.append(df)


prices: 0
[]
url_product: 0
[]


/var/folders/9z/sjrbpv9j1996t3_4wv3ry65c0000gn/T/ipykernel_65512/2389928353.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aggregated=df_aggregated.append(df)


---------------


In [197]:
df_wine_diplo

,0,1,country
0,42.05,"Domaine Cailbourdin, Pouilly-Sur-Loire, Pouill...",france
1,75.6,"Maison Antoine Geoffroy, Beaune, Gevrey Chambe...",france
2,148.85,"Domaine Daniel-Etienne Defaix, Chablis Grand C...",france
3,75.6,"Domaine Daniel-Etienne Defaix, Chablis Vieille...",france
4,3139.59,"2016 Château Duhart-Milon Rothschild, 4eme Cru...",france
...,...,...,...
6,12.65,Château Ksara Blanc de Blancs,rest-of-the-world
7,11.5,Château Ksara Sunset Rosé,rest-of-the-world
8,9.05,"Vinho Verde Loureiro/Alvarinho, Azevedo, Portugal",rest-of-the-world
9,6.8,Mateus Rosé,rest-of-the-world


In [198]:
df_wine_diplo['vivino_url']=''
df_wine_diplo['vivino_rating']=''

In [199]:
df_wine_diplo

,0,1,country,vivino_url,vivino_rating
0,42.05,"Domaine Cailbourdin, Pouilly-Sur-Loire, Pouill...",france,,
1,75.6,"Maison Antoine Geoffroy, Beaune, Gevrey Chambe...",france,,
2,148.85,"Domaine Daniel-Etienne Defaix, Chablis Grand C...",france,,
3,75.6,"Domaine Daniel-Etienne Defaix, Chablis Vieille...",france,,
4,3139.59,"2016 Château Duhart-Milon Rothschild, 4eme Cru...",france,,
...,...,...,...,...,...
6,12.65,Château Ksara Blanc de Blancs,rest-of-the-world,,
7,11.5,Château Ksara Sunset Rosé,rest-of-the-world,,
8,9.05,"Vinho Verde Loureiro/Alvarinho, Azevedo, Portugal",rest-of-the-world,,
9,6.8,Mateus Rosé,rest-of-the-world,,


In [208]:
# Code to send a wine name to Vivino and scrape the first result


vivino_url_wine=[]
vivino_rating_wine=[]

list_wine_search=df_wine_diplo[1]

# open browser
browser = webdriver.Chrome(ChromeDriverManager().install())

#for each wine name in the list 
for search_txt in list_wine_search:
    # write the wine name
    browser.get('https://www.vivino.com/FR/en/')  
    
    try:
        # find the input bar
        element=browser.find_element_by_class_name("searchBar_searchInput__Jje-K")
        element.clear()
        element.send_keys(search_txt)
        element.send_keys(Keys.RETURN)

        # get the first result
        time.sleep(2)

        htmlsource = browser.page_source
        rating_regexp="\"ratingValue\":(.*?),"
        url_regexp='\"Product\",\"\@id\":\"(.*?)\",'

        time.sleep(1)

        vivino_url_wine_inter=[re.findall(url_regexp, htmlsource)[0]]
        vivino_rating_wine_inter= [re.findall(rating_regexp, htmlsource)[0]]

        vivino_url_wine=vivino_url_wine+vivino_url_wine_inter
        vivino_rating_wine=vivino_rating_wine+vivino_rating_wine_inter
    
    except:
        # so the same thing but regenerate the browser and wait 60s
        print('blocked - waiting 60')
        browser.close()
        time.sleep(60)
        
        browser = webdriver.Chrome(ChromeDriverManager().install())
        browser.get('https://www.vivino.com/FR/en/')  
        
        element=browser.find_element_by_class_name("searchBar_searchInput__Jje-K")
        element.clear()
        element.send_keys(search_txt)
        element.send_keys(Keys.RETURN)

        # get the first result
        time.sleep(2)

        htmlsource = browser.page_source
        rating_regexp="\"ratingValue\":(.*?),"
        url_regexp='\"Product\",\"\@id\":\"(.*?)\",'

        time.sleep(1)

        vivino_url_wine_inter=[re.findall(url_regexp, htmlsource)[0]]
        vivino_rating_wine_inter= [re.findall(rating_regexp, htmlsource)[0]]

        vivino_url_wine=vivino_url_wine+vivino_url_wine_inter
        vivino_rating_wine=vivino_rating_wine+vivino_rating_wine_inter
        
    

blocked - waiting 60
blocked - waiting 60
blocked - waiting 60
blocked - waiting 60


In [215]:
df_wine_diplo['vivino_url']=vivino_url_wine

In [216]:
df_wine_diplo['vivino_rating']=vivino_rating_wine

In [218]:
df_wine_diplo.to_csv('wine_diplo_2023.csv')